In [1]:
import numpy as np
import pandas as pd

import re, nltk

import string

import ssl
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from collections import Counter

from textblob import TextBlob
from spellchecker import SpellChecker

In [2]:
df = pd.read_csv('C:/Users/Crolw/Downloads/twcs.csv')

## Change Datatypes to correct type
#### author_id: object -> string
#### text: object -> string
#### created_at: object -> string

In [3]:
df['author_id'] = df['author_id'].astype('string')
df['text'] = df['text'].astype('string')
df['created_at'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d').astype('string')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                string 
 2   inbound                  bool   
 3   created_at               string 
 4   text                     string 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(1), string(3)
memory usage: 131.4+ MB


In [5]:
df.isna().any()

tweet_id                   False
author_id                  False
inbound                    False
created_at                 False
text                       False
response_tweet_id           True
in_response_to_tweet_id     True
dtype: bool

### Text Cleaning

#### Make new variable to focus on text data

In [6]:
# Since we are only interested in text the part for now
textDF = df[['text']]
print('shape is:', textDF.shape)
textDF.head()

shape is: (2811774, 1)


,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


#### Make new column to convert text into only lowercase

In [7]:
textDF['text_lower'] = textDF.loc[:,'text'].str.lower()
textDF.head(10)

C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\816064500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_lower'] = textDF.loc[:,'text'].str.lower()


,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...
9,@115713 We understand your concerns and we'd l...,@115713 we understand your concerns and we'd l...


#### Make new column with no puncuations

In [8]:
punctuations = string.punctuation
print(punctuations)

%timeit
def remove_punctuations(text):
    return text.translate(str.maketrans('','',punctuations))
    

textDF['text_wo_punct'] = textDF['text_lower'].apply(lambda text: remove_punctuations(text))
textDF.head(10)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\438778996.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_punct'] = textDF['text_lower'].apply(lambda text: remove_punctuations(text))


,text,text_lower,text_wo_punct
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...,sprintcare you gonna magically change your con...
9,@115713 We understand your concerns and we'd l...,@115713 we understand your concerns and we'd l...,115713 we understand your concerns and wed lik...


In [9]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [10]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Load stopwords

In [11]:
', '.join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

#### Make new column that eliminates stopwords

In [12]:
stopwords_set = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join(words for words in str(text).split() if words not in stopwords_set)
    

textDF['text_wo_stop'] = textDF['text_wo_punct'].apply(lambda text: remove_stopwords(text))
textDF.head()

C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\1961305086.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_stop'] = textDF['text_wo_punct'].apply(lambda text: remove_stopwords(text))


,text,text_lower,text_wo_punct,text_wo_stop
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare


#### Count most used words

In [13]:
cnt = Counter()

for text in textDF['text_wo_stop'].values:
    for word in text.split():
        cnt[word] +=1
        
cnt.most_common(10)

[('us', 451262),
 ('please', 402709),
 ('dm', 335374),
 ('help', 267633),
 ('hi', 224603),
 ('thanks', 206452),
 ('get', 200374),
 ('sorry', 192246),
 ('like', 146385),
 ('know', 145407)]

#### Make new column with frequently used words eliminated

In [14]:
freq_words = set([w for (w,wc) in cnt.most_common(10)])

def remove_freqwords(text):
    return ' '.join(word for word in str(text).split() if word not in freq_words)

textDF['text_wo_freq'] = textDF['text_wo_stop'].apply(lambda text:remove_freqwords(text))
textDF.head()

C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\3239245677.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_freq'] = textDF['text_wo_stop'].apply(lambda text:remove_freqwords(text))


,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare


#### Make new column with rare words eliminated

In [15]:
n_rare_words = 10
rare_words = set([w for (w,c) in cnt.most_common()][:- n_rare_words: -1])
print(rare_words)

def remove_rarewords(text):
    return ' '.join(word for word in str(text).split() if word not in rare_words)

textDF['text_wo_freqrare'] = textDF['text_wo_freq'].apply(lambda text:remove_rarewords(text))
textDF.head()

{'823868', '823867', 'httpstcov2tmhetl7q', '823869', 'httpstco7uqpwyh1b6', 'httpstco4v1ft0th5x', 'notjustxmasallyearround', 'httpstcogfyuq1kjtk', '823870'}


C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\1870184822.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_freqrare'] = textDF['text_wo_freq'].apply(lambda text:remove_rarewords(text))


,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq,text_wo_freqrare
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare


#### Stemming

In [16]:
stemmer = PorterStemmer()

In [17]:
# Stem every word
textDF['text_stemmed'] = textDF['text_lower'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))

In [18]:
textDF.head(10)

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq,text_wo_freqrare,text_stemmed
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,@ 115712 i understand . i would like to assist...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,@ sprintcar and how do you propos we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,@ sprintcar i have sent sever privat messag an...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,@ 115712 pleas send us a privat messag so that...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,@ sprintcar i did .
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...,115712 please send us private message gain det...,115712 send private message gain details account,115712 send private message gain details account,@ 115712 can you pleas send us a privat messag...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcare worst customer service,@ sprintcar is the worst custom servic
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm look kc,115713 saddening hear shoot look kc,115713 saddening hear shoot look kc,@ 115713 thi is sadden to hear . pleas shoot u...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...,sprintcare you gonna magically change your con...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,@ sprintcar you gon na magic chang your connec...
9,@115713 We understand your concerns and we'd l...,@115713 we understand your concerns and we'd l...,115713 we understand your concerns and wed lik...,115713 understand concerns wed like please sen...,115713 understand concerns wed send direct mes...,115713 understand concerns wed send direct mes...,@ 115713 we understand your concern and we 'd ...


#### Lemmatization

In [19]:
lemmatizer = WordNetLemmatizer()

In [20]:
# Lemmanize every word
textDF['text_lemmatized'] = textDF['text_lower'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

In [21]:
textDF.head(10)

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq,text_wo_freqrare,text_stemmed,text_lemmatized
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,@ 115712 i understand . i would like to assist...,@ 115712 i understand . i would like to assist...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,@ sprintcar and how do you propos we do that,@ sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,@ sprintcar i have sent sever privat messag an...,@ sprintcare i have sent several private messa...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,@ 115712 pleas send us a privat messag so that...,@ 115712 please send u a private message so th...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,@ sprintcar i did .,@ sprintcare i did .
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...,115712 please send us private message gain det...,115712 send private message gain details account,115712 send private message gain details account,@ 115712 can you pleas send us a privat messag...,@ 115712 can you please send u a private messa...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcare worst customer service,@ sprintcar is the worst custom servic,@ sprintcare is the worst customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm look kc,115713 saddening hear shoot look kc,115713 saddening hear shoot look kc,@ 115713 thi is sadden to hear . pleas shoot u...,@ 115713 this is saddening to hear . please sh...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...,sprintcare you gonna magically change your con...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,@ sprintcar you gon na magic chang your connec...,@ sprintcare you gon na magically change your ...
9,@115713 We understand your concerns and we'd l...,@115713 we understand your concerns and we'd l...,115713 we understand your concerns and wed lik...,115713 understand concerns wed like please sen...,115713 understand concerns wed send direct mes...,115713 understand concerns wed send direct mes...,@ 115713 we understand your concern and we 'd ...,@ 115713 we understand your concern and we 'd ...


#### Removal of Emojis and Emoticons

In [22]:
emojiPattern = re.compile("["u"\U0001F600-\U0001F64F""]+", flags=re.UNICODE)

In [23]:
textDF['text_wo_emoticons'] = textDF['text_lower'].apply(lambda x: emojiPattern.sub(r'', x))

#### Conversion of Emoticons and Emojis to Words

#### Removal of URLs

In [24]:
textDF["text_wo_url"] = textDF["text_lower"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()

C:\Users\Crolw\AppData\Local\Temp\ipykernel_4112\4193889894.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  textDF["text_wo_url"] = textDF["text_lower"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()


In [25]:
textDF.head(10)

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq,text_wo_freqrare,text_stemmed,text_lemmatized,text_wo_emoticons,text_wo_url
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,@ 115712 i understand . i would like to assist...,@ 115712 i understand . i would like to assist...,@115712 i understand. i would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,@ sprintcar and how do you propos we do that,@ sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,@ sprintcar i have sent sever privat messag an...,@ sprintcare i have sent several private messa...,@sprintcare i have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,@ 115712 pleas send us a privat messag so that...,@ 115712 please send u a private message so th...,@115712 please send us a private message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,@ sprintcar i did .,@ sprintcare i did .,@sprintcare i did.,@sprintcare i did.
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...,115712 please send us private message gain det...,115712 send private message gain details account,115712 send private message gain details account,@ 115712 can you pleas send us a privat messag...,@ 115712 can you please send u a private messa...,@115712 can you please send us a private messa...,@115712 can you please send us a private messa...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcare worst customer service,@ sprintcar is the worst custom servic,@ sprintcare is the worst customer service,@sprintcare is the worst customer service,@sprintcare is the worst customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm look kc,115713 saddening hear shoot look kc,115713 saddening hear shoot look kc,@ 115713 thi is sadden to hear . pleas shoot u...,@ 115713 this is saddening to hear . please sh...,@115713 this is saddening to hear. please shoo...,@115713 this is saddening to hear. please shoo...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...,sprintcare you gonna magically change your con...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,@ sprintcar you gon na magic chang your connec...,@ sprintcare you gon na magically change your ...,@sprintcare you gonna magically change you

#### Removal of HTML Tags

In [26]:
textDF['text_wo_tags'] = textDF['text_lower'].apply(lambda x: ' '.join([re.sub('<.*?>','', x)]))

In [27]:
textDF.head(10)

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_freq,text_wo_freqrare,text_stemmed,text_lemmatized,text_wo_emoticons,text_wo_url,text_wo_tags
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,@ 115712 i understand . i would like to assist...,@ 115712 i understand . i would like to assist...,@115712 i understand. i would like to assist y...,@115712 i understand. i would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,@ sprintcar and how do you propos we do that,@ sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,@ sprintcar i have sent sever privat messag an...,@ sprintcare i have sent several private messa...,@sprintcare i have sent several private messag...,@sprintcare i have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,@ 115712 pleas send us a privat messag so that...,@ 115712 please send u a private message so th...,@115712 please send us a private message so th...,@115712 please send us a private message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,@ sprintcar i did .,@ sprintcare i did .,@sprintcare i did.,@sprintcare i did.,@sprintcare i did.
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...,115712 please send us private message gain det...,115712 send private message gain details account,115712 send private message gain details account,@ 115712 can you pleas send us a privat messag...,@ 115712 can you please send u a private messa...,@115712 can you please send us a private messa...,@115712 can you please send us a private messa...,@115712 can you please send us a private messa...
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcare worst customer service,@ sprintcar is the worst custom servic,@ sprintcare is the worst customer service,@sprintcare is the worst customer service,@sprintcare is the worst customer service,@sprintcare is the worst customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm look kc,115713 saddening hear shoot look kc,115713 saddening hear shoot look kc,@ 115713 thi is sadden to hear . pleas shoot u...,@ 115713 this is saddening to hear . please sh...,@115713 this is saddening to hear. please shoo...,@115713 this is saddening to hear. please shoo...,@115713 this is saddening to hear. please shoo...
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna 

#### Chat Words Conversion

#### Spelling Correction

In [ ]:
textDF['text_correction'] = textDF['text_lower'].apply(lambda x: ' '.join([SpellChecker().correction(word) for word in word_tokenize(x)]))

In [ ]:
textDF.head(10)

### Data Wrangling

In [ ]:
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]

QnR = pd.merge(first_inbound, df, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')

# Filter to only outbound replies (from companies)
QnR = QnR[QnR.inbound_y ^ True]
print(f'Data shape: {QnR.shape}')
QnR.head()